In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle
import os
import json
from collections import defaultdict
from pycocotools.coco import COCO
from tqdm import tqdm
from shutil import copyfile
import csv
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile
import cv2


In [0]:
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import *
from keras import backend as K

In [0]:
# Download caption annotation files 20GB files took around 20 mins to run

annotation_folder = '/annotations/'
if not os.path.exists(os.path.abspath('.') + annotation_folder):
  annotation_zip = tf.keras.utils.get_file('captions.zip',
                                          cache_subdir=os.path.abspath('.'),
                                          origin = 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip',
                                          extract = True)
  annotation_file = os.path.dirname(annotation_zip)+'/annotations/captions_train2017.json'
  os.remove(annotation_zip)

# Download image files
image_folder = '/train2014/'
if not os.path.exists(os.path.abspath('.') + image_folder):
  image_zip = tf.keras.utils.get_file('train2017.zip',
                                      cache_subdir=os.path.abspath('.'),
                                      origin = 'http://images.cocodataset.org/zips/train2017.zip',
                                      extract = True)
  PATH = os.path.dirname(image_zip) + image_folder
  os.remove(image_zip)
else:
  PATH = os.path.abspath('.') + image_folder

  # Paths
captions_path = '/content/annotations/captions_train2017.json'
annotations_path = '/content/annotations/instances_train2017.json'
images_path = '/content/train2017/'

dir_content = os.listdir(images_path)

# Load captions
with open(captions_path) as json_file: 
    captions = json.load(json_file)

# Load anotations
with open(annotations_path) as json_file: 
    annotations = json.load(json_file)



19336863744/19336861798 [==============================] - 310s 0us/step


In [0]:

#Map each category with the image_ids in a dictionary format
#where keys are category number and values are image ids for that category.

image_filenames = defaultdict(list)
category_animal_id = [3,5,17,18]
category_animal_name = ['vehicle','plane','cat','dog']

category_id = category_animal_id
category_name = category_animal_name

category_map = dict()
for i in range(len(category_id)):
    category_map[category_id[i]] = category_name[i]

for i in annotations['annotations']:
    if i['category_id'] in category_id:
        image_filenames[i['category_id']].append(i['image_id'])

print('Completed')

Completed


In [0]:
#Task is to map image_ids with image_filenames

plane = []
plane.extend(image_filenames[5])
f_plane = []
for i in range(len(captions['images'])):
  if(captions['images'][i]['id'] in plane ):
    f_plane.append(captions['images'][i]['file_name'])

print(len(f_plane))

2986


In [0]:

def plotCleanedImages(images_name,idx_to_start):
  #Plotting few plane images to verify it's clean as expected
  for img in images_name[idx_:320]:
    img_ = cv2.imread(os.path.join(images_path,img))
    plt.subplots(1,1)
    plt.imshow(img_)
    idx_to_start +=1

#Plot plane images
plotCleanedImages(f_planes,300)

In [0]:
#As images are pretty clean, we randomly select 1200 without repition images and made a dataframe from export to drive

random_indexes = random.sample(range(len(f_plane)),1200)

retain_images_plane = []
for i in random_indexes:
  retain_images_plane.append(f_plane[i])

#Making dataframe to export

df_plane = pd.DataFrame(retain_images_plane,columns=['Images'])
df_plane['Label'] = ['plane'] * len(df_plane)
df_plane.to_csv('Cleaned_Plane.csv',index=False)

#Exporting
!cp 'Cleaned_Plane.csv' "drive/My Drive/"

#In this way we finished extracting cleaner images for plane category

1200

In [0]:
#Loading the pretrained model
model = VGG16(weights='imagenet',include_top=True)

553467904/553467096 [==============================] - 6s 0us/step


In [0]:


#Get images names for dog and cat categories using same tactic as plane

dogcat = []
dogcat.extend(image_filenames[17])
dogcat.extend(image_filenames[18])
f_dc = []
for i in range(len(captions['images'])):
  if(captions['images'][i]['id'] in dogcat ):
    f_dc.append(captions['images'][i]['file_name'])

#Performing for Cars
cars = []
cars.extend(image_filenames[3])
f_cars = []
for i in range(len(captions['images'])):
  if(captions['images'][i]['id'] in cars ):
    f_cars.append(captions['images'][i]['file_name'])

count = 0
retain_images = [] #store image to retain
label_images = []  #store category of retained image

#Kind of filter applied -- if pretrained model predicts anyone of these keywords as classes in an image retain that image else discard
labels = ['cat','dog','husky','kitten','pug','puppies']
retain_cars = ['streetcar','limousine','cab','passenger_car','car_mirror','jeep','car','minivan']

#Check function perform many to one mapping i.e pug,puppies,husky will be tagged as dog

def check(name):

  #Checking for cars
  if(name in retain_cars):
    label_images.append('Vehicle')
    return True

  #Checking for cat and dog
  for i in labels:
    if(i in name):
      if( i == 'cat' or i =='kitten'):
        label_images.append('cat')
      else:
        label_images.append('dog')
      return True

  return False

#This loop read the images present in f_dc lists and pass to VGG16 to make predictions
#Predictions are then decoded and top 1 predicted classes is taken into account as it has highest probability
#If the predicted class statisfies the check function the image name get append into retain_images list
#We have monitored closely in batches of 100 how many images the model has captured perfectly

def getPredictionsToStore(f_names,count):
  for idx in range(len(f_names)):
      img_path = os.path.join(images_path,f_names[idx])
      img = cv2.imread(img_path)
      img = cv2.resize(img, (224,224),cv2.IMREAD_GRAYSCALE,interpolation=cv2.INTER_CUBIC)
      img_data = image.img_to_array(img)
      img_data = np.expand_dims(img_data, axis=0)
      img_data = preprocess_input(img_data)
      predictions = model.predict(img_data)
      classes = decode_predictions(predictions,top=1)
      name,like = classes[0][0][1],classes[0][0][2] 
      if(check(name)):
        retain_images.append(f_names[idx])
      count +=1
      if(count % 100 == 0):
        print(f'Count is {count} and images captured is {len(retain_images)}')

  print('Completed')

print('Capturing for Cat_Dog Categories')
getPredictionsToStore(f_dc,0)

print('Capturing for Car categories')
getPredictionsToStore(f_cars,0)

#As seen in the output
#The acceptance rate is 10-15% per 100 images, this help to process only clean data with actual label as cat/dog,rather than arbitary images seen in COCO dataset labeled as cat/dog

Count is 100 and images captured is 14
Count is 200 and images captured is 27
Count is 300 and images captured is 35
Count is 400 and images captured is 51
Count is 500 and images captured is 59
Count is 600 and images captured is 75
Count is 700 and images captured is 83
Count is 800 and images captured is 94
Count is 900 and images captured is 103
Count is 1000 and images captured is 114
Count is 1100 and images captured is 129
Count is 1200 and images captured is 144
Count is 1300 and images captured is 154
Count is 1400 and images captured is 169
Count is 1500 and images captured is 180
Count is 1600 and images captured is 189
Count is 1700 and images captured is 204
Count is 1800 and images captured is 216
Count is 1900 and images captured is 228
Count is 2000 and images captured is 240
Count is 2100 and images captured is 253
Count is 2200 and images captured is 266
Count is 2300 and images captured is 274
Count is 2400 and images captured is 289
Count is 2500 and images captured

In [0]:
#Check the cleaned images and predicted category correctly by VGG16 for each categories
plotCleanedImages(retain_images,10)

In [0]:
#Convert into dataframe and ready for export for all categories

from google.colab import drive
import pandas as pd

df_dcc = pd.DataFrame(retain_images,columns=['Images'])
df_dcc['Label'] = label_images

#Combining all four categories dataframe and merging into one dataframe for export
df_images = pd.concat([df_plane,df,df_dcc])
df_images.to_csv('Cleaned_Images.csv',index=False)

#Export Clean images to Gdrive
!cp 'Cleaned_Images.csv' "drive/My Drive/"

In [0]:
#From merge dataFrame get the images label and images names

#f_cars = df_images[df_images['Label'] == 'Vehicle']
#f_cat = df_images[df_images['Label'] == 'cat']
#f_dog = df_images[df_images['Label'] == 'dog']
#f_plane = df_images[df_images['Label'] == 'plane']

df_unique = pd.DataFrame()
df_unique = df_images.drop_duplicates(keep=False)

def getImage_Labels():

  #Remove the duplicates in case in the dataframe to avoid mapping same images to multiple category
  f_car = df_unique[df_unique['Label'] == 'Vehicle']['Images']
  f_cat = df_unique[df_unique['Label'] == 'cat']['Images']
  f_dog = df_unique[df_unique['Label'] == 'dog']['Images']
  f_plane = df_unique[df_unique['Label'] == 'plane']['Images']

  #Type cast to list
  f_car = list(f_car)
  f_cat = list(f_cat)
  f_dog = list(f_dog)
  f_plane = list(f_plane)

  return f_car,f_cat,f_dog,f_plane

f_car,f_cat,f_dog,f_plane = getImage_Labels()

In [0]:
#This process is extracting from image name get their image id
#This will be used to extract the captions for that image as the field used in annotations is 'image_id'

car_ids =  []
dog_ids = []
plane_ids = []
cat_ids = []
dict_images = defaultdict(list)

for i in range(len(captions['images'])):
  file_name = captions['images'][i]['file_name']
  if( file_name in f_car):
    car_ids.append(captions['images'][i]['id'])
  elif(file_name in f_cat):
    cat_ids.append(captions['images'][i]['id'])
  elif(file_name in f_dog):
    dog_ids.append(captions['images'][i]['id'])
  elif(file_name in f_plane):
    plane_ids.append(captions['images'][i]['id'])
 

In [0]:
#Re-initialzing the original image_filenames dict with image ids uniquely belonging to particular category

category_vehicle_id = [3, 5]
category_vehicle_name = ['car', 'plane']
category_animal_id = [17, 18]
category_animal_name = ['cat', 'dog']
category_id = category_vehicle_id + category_animal_id
category_name = category_vehicle_name + category_animal_name
category_map = dict()

for i in range(len(category_id)):
    category_map[category_id[i]] = category_name[i]

image_filenames_all = {3:car_ids,5:plane_ids,17:cat_ids,18:dog_ids}


In [0]:
image_cat_file_counts = defaultdict(list)
for i in tqdm(category_id):
    for j in tqdm(image_filenames_all[i]):
        count_cats = 0
        for k in annotations['annotations']:
            if k['image_id'] == j:
                count_cats += 1
        image_cat_file_counts[i].append([j, count_cats])

In [0]:
# Removing duplicates
unique_dict = {}
for k,v in image_cat_file_counts.items():
    new=[]
    for i in v:
        new.append(tuple(i))
    unique_dict[k] = list(set(new))

# Saving images and captions to directory
coco_caps = COCO(captions_path)

In [0]:
# Filtering the image filenames

filtered_filenames_train = defaultdict(list)
filtered_filenames_test = defaultdict(list)

for k, v in unique_dict.items():
    v_sorted = sorted(v, key = (lambda x: x[1]))
    for i in range(len(v_sorted)):
        if i%5==0:            
            filtered_filenames_test[k].append(v_sorted[i])
        else:
            filtered_filenames_train[k].append(v_sorted[i])

#From the 2945 images we split into train and test by creating respective diretories

for i in category_name:
    os.makedirs('/content/train/'+i)
    os.makedirs('/content/test/'+i)

In [0]:
# WRITE TRAIN
captions_train = []
for k, v in filtered_filenames_train.items():
    for file in tqdm(v):

        # Save Image
        dirname = category_map[k]
        filename = str(file[0]).zfill(12)+'.jpg'
        src = images_path + filename

        dst = 'content'+'/train/'+dirname+'/'+filename
        copyfile(src, dst)
        
        #extract captions
        annIds = coco_caps.getAnnIds(imgIds=file[0])
        anns = coco_caps.loadAnns(annIds)
        
        for i in range(len(anns)):
            captions_train.append([filename+'#'+str(i), anns[i]['caption']])
    
with open('/content/train/captions_train.csv', "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(captions_train)

100%|██████████| 294/294 [00:00<00:00, 353.99it/s]


In [0]:
for i in os.listdir('/content/train'):
  if( i == 'captions_train.csv'):
    df_captions = pd.read_csv('/content/train/captions_train.csv',names=['Images','Labels'])

In [0]:
df_captions.head(10)

,Images,Labels
0,000000423052.jpg#0,A sign and car on a city street.
1,000000423052.jpg#1,Store front on an old narrow city street with ...
2,000000423052.jpg#2,A sidewalk with a large sign that's been alter...
3,000000423052.jpg#3,"a vandalized parking sign that reads ""free con..."
4,000000423052.jpg#4,A sign points to a traffic cone above it.
5,000000407559.jpg#0,Scene of a building through the mirror of a ve...
6,000000407559.jpg#1,Rear view mirror of car with a sign reflected ...
7,000000407559.jpg#2,City lights shine while a car rear view mirror...
8,000000407559.jpg#3,Someone checks a side mirror while in their car.
9,000000407559.jpg#4,There are some colored lights hanging from str...


In [0]:
# WRITE TEST
captions_test = []
for k, v in filtered_filenames_test.items():
    for file in tqdm(v):
        # Save Image
        dirname = category_map[k]
        filename = str(file[0]).zfill(12)+'.jpg'
        src = images_path + filename
        dst = '/content/test/'+dirname+'/'+filename
        copyfile(src, dst)
        #extract captions
        annIds = coco_caps.getAnnIds(imgIds=file[0])
        anns = coco_caps.loadAnns(annIds)
        
        for i in range(len(anns)):
            captions_test.append([filename+'#'+str(i), anns[i]['caption']])

with open('/content/test/captions_test.csv', "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(captions_test)

100%|██████████| 74/74 [00:00<00:00, 1240.12it/s]


In [0]:
#Exporting the captions for train and test to Gdrive along with a DataFrame which contains image_name and label it belonged to.
df_unique.to_csv('Unique_images_with_Labels.csv',header=True)
!cp 'Unique_images_with_Labels.csv' "drive/My Drive/"
!cp '/content/test/captions_test.csv' "drive/My Drive/"
!cp '/content/train/captions_train.csv' "drive/My Drive/"


**Thank you**